In [1]:
!pip install -q condacolab
!git clone --single-branch --branch cecam https://github.com/cmelab/hoomd-organics
import condacolab
condacolab.install()

Cloning into 'hoomd-organics'...
remote: Enumerating objects: 3573, done.
remote: Counting objects: 100% (829/829), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 3573 (delta 674), reused 646 (delta 646), pack-reused 2744 (from 1)
Receiving objects: 100% (3573/3573), 1.53 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (2461/2461), done.
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [1]:
#!pip install --upgrade ipykernel #breaks things?
import os
os.chdir("hoomd-organics")
!mamba env update -n base -f environment-cpu.yml
!python -m pip install -e .
import warnings
warnings.filterwarnings('ignore')

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



cudatoolkit-11.8.0   | 682.5 MB  | :   0% 0/1 [00:00<?, ?it/s]
gcc_impl_linux-64-14 | 70.1 MB   | :   0% 0/1 [00:00<?, ?it/s]

pillow-11.1.0        | 40.1 MB   | :   0% 0/1 [00:00<?, ?it/s]


libllvm14-14.0.6     | 30.0 MB   | :   0% 0/1 [00:00<?, ?it/s]



libflint-3.1.3.1     | 20.2 MB   | :   0% 0/1 [00:00<?, ?it/s]




rdkit-2024.09.5      | 19.0 MB   | :   0% 0/1 [00:00<?, ?it/s]





parmed-4.3.0         | 18.5 MB   | :   0% 0/1 [00:00<?, ?it/s]






scipy-1.15.2         | 16.4 MB   | :   0% 0/1 [00:00<?, ?it/s]







libclang-cpp14-14.0. | 15.8 MB   | :   0% 0/1 [00:00<?, ?it/s]








pandas-2.2.3         | 15.0 MB   | :   0% 0/1 [00:00<?, ?it/s]









sysroot_linux-64-2.1 | 14.5 MB 

In [5]:
import hoomd
# Attach to CPU and create simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_gsd(filename='random.gsd') #N and V are set here

#Potential and integrator setup
integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer = 0.4)
lj_potential = hoomd.md.pair.LJ(nlist=cell)
lj_potential.params[('A','A')] = dict(epsilon=1,sigma=1)
lj_potential.r_cut[('A','A')]=2.5
nvt = hoomd.md.methods.NVT(kT=1.5,filter=hoomd.filter.All(),tau=1.)
integrator.forces.append(lj_potential)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

# Set the simulation state
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

# Run  the simulation  (a few seconds)
sim.run(1e4)

RuntimeError: GSD: Bad file descriptor - random.gsd

In [ ]:
#And finally let's visualize the output (usually ~4 seconds, but sometime slow on /shared?)
import mbuild
hoomd.write.GSD.write(state=sim.state, filename='final.gsd', mode='wb')
final = mbuild.load("final.gsd")

In [ ]:
view = final.visualize()
view.setStyle({"sphere": {"scale": 3., "colorscheme": {"A":"red"} }})
view.addBox({"center":{"x":0,"y":0,"z":0},"dimensions": {"w":10*final.box.Lx,"h":10*final.box.Ly,"d":10*final.box.Lz}})
view.show()